## Using Input Parameters to efficiently re-run simulations with different parameters

The previous notebook described the PyBaMM pipeline and how you can use the `pybamm.Simulation` class to efficiently run simulations. Recall the pipeline was:

1. Parameter replacement
2. Discretisation
3. Solver setup
4. Solver solve
5. Post-processing

An obvious question is how can we efficiently re-run simulations with different parameters (step 1 in the pipeline)? Ideally we'd like to do this without having to re-build the model and discretisation each time, which as we've seen is a costly process.

In [1]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import numpy as np

Note: you may need to restart the kernel to use updated packages.



## Using Input Parameters

The answer is to use input parameters and the [`pybamm.InputParameter`](https://docs.pybamm.org/en/stable/source/api/expression_tree/input_parameter.html) class. Input parameters are a special type of parameter that is *not* replaced by a value during step 1 of the pipeline, and can be used to replace many of the parameters in the model (as we'll see later this does not apply to some geometric parameters). 

If you use the `pybamm.ParameterVariables` class to set the parameters of your model, you can set any parameter in the model to be an input parameter by updating its parameter value to be the string `[input]`. For example, to set the current in the SPM model to be an input parameter, you can do the following:

In [2]:
model = pybamm.lithium_ion.SPM()
parameter_values = model.default_parameter_values
parameter_values["Current function [A]"] = "[input]"


If you are building up a model from scratch, you can also use `pybamm.InputParameter` directly to create an input parameter. For example, the code below creates an exponential decay model with a decay constant as an input parameter:

In [3]:
model = pybamm.BaseModel()
k = pybamm.InputParameter("k")
x = pybamm.Variable("x")
model.rhs = {x: -k * x}

## Example

Let's see how we can use input parameters to efficiently re-run simulations with different parameters. We'll start with a script that loops over the current to solve the model, and then show how we can use input parameters to do this more efficiently.

In [4]:
import time

average_time = 0
n = 9
model = pybamm.lithium_ion.SPM()
solver = pybamm.IDAKLUSolver()
params = model.default_parameter_values
for current in np.linspace(-1.1, 1.0, n):
    time_start = time.perf_counter()
    params["Current function [A]"] = current
    sim = pybamm.Simulation(model, solver=solver, parameter_values=params)
    sol = sim.solve([0, 3600])
    t_evals = np.linspace(0, 3600, 100)
    voltage = sol["Terminal voltage [V]"](t_evals)
    time_end = time.perf_counter()
    average_time += time_end - time_start
print(f"Average time taken: {average_time / n:.3f} seconds")

Average time taken: 0.205 seconds


The most important thing to note about this script is that we are creating the simulation object on every iteration of this loop. This means we are running though all the steps of the pipeline, rebuilding the model and performing the discretisations, at each iteration of the loop. We do this even though most of the structure of the model, and in particular the numerical discretisations of the spatial gradients, is unchanged. As we've seen previously, the discretisation step is often the most expensive part of the pipeline, so we'd like to avoid repeating it if possible.

Now let's see how we will use input parameters to do this more efficiently. To do this we will move the simulation object creation outside of the loop, and use an input parameter for the current instead.

In [5]:
average_time = 0
n = 10
model = pybamm.lithium_ion.SPM()
solver = pybamm.IDAKLUSolver()
params = model.default_parameter_values
params["Current function [A]"] = "[input]"
sim = pybamm.Simulation(model, solver=solver, parameter_values=params)
for current in np.linspace(0.1, 1.0, n):
    time_start = time.perf_counter()
    sol = sim.solve([0, 3600], inputs={"Current function [A]": current})
    t_evals = np.linspace(0, 3600, 100)
    voltage = sol["Terminal voltage [V]"](t_evals)
    time_end = time.perf_counter()
    average_time += time_end - time_start
print(f"Average time taken: {average_time / n:.3f} seconds")

Average time taken: 0.047 seconds


You can see that we've improved the performance of our loop significantly by moving the creation of the simulation object outside of the loop, thus avoiding the need to rebuild the model from scratch at each iteration. 


## Limitations

### Increased solve time

This is usually a minor effect, but adding input parameters to a model can sometimes increase the time taken to solve the model, due to the increased complexity of the model.

It is interesting to compare the time taken to solve the model with an input parameter to the time taken to solve the original model. There is a good reason why step 1 of the pipeline replaces parameters with values: this allows PyBaMM to simplify the equations significantly by combining terms and performing operations between values, rather than between symbols. E.g. if your model has parameters `a` and `b` and has the equation `a*b` somewhere in the model, if both `a` and `b` are replaced by values, PyBaMM can simplify this to `c`, where `c = a*b`. If either `a` or `b` is an input parameter, PyBaMM cannot simplify this equation, and will have to evaluate `a*b` at each time step of the simulation. 

See for example the following code that adds three input parameters to the DFN model, the additional time taken to solve the model is very small, but if you have a costly RHS evaluation, this could be more significant.

In [ ]:
model = pybamm.lithium_ion.DFN()
params = model.default_parameter_values
solver = pybamm.IDAKLUSolver()
sim = pybamm.Simulation(model, solver=solver)
sol = sim.solve([0, 3600])
start_time = time.perf_counter()
n = 1000
for _ in range(n):
    sol = sim.solve([0, 3600])
end_time = time.perf_counter()
print(f"Time taken without input parameter: {(end_time - start_time) / n:.4f} seconds")


model = pybamm.lithium_ion.DFN()
solver = pybamm.IDAKLUSolver()
param_names = [
    "Current function [A]",
    "Negative electrode active material volume fraction",
    "Nominal cell capacity [A.h]",
]
inputs = {name: params[name] for name in param_names}
for param_name in param_names:
    params[param_name] = "[input]"
sim = pybamm.Simulation(model, solver=solver, parameter_values=params)
sol = sim.solve([0, 3600], inputs=inputs)
start_time = time.perf_counter()
for _ in range(n):
    sol = sim.solve([0, 3600], inputs=inputs)
end_time = time.perf_counter()
print(f"Time taken with input parameter: {(end_time - start_time) / n:.4f} seconds")

Time taken without input parameter: 0.0409 seconds
Time taken with input parameter: 0.0411 seconds


In most cases, the addition of an input parameter will only slightly increase the complexity of the model and the solve time. However, this can add up for many different input parameters, so you should only use them if neccessary.



### Geometric parameters cannot be input parameters

Input parameters cannot be used for parameters that are used during the descretisation step. These are generally parameters affecting the geometry, such as the electrode or separator thicknesses. If you try to use an input parameter for a parameter that affects the discretisation, you will get an error, most likely during the discretisation step. For example, these are the parameters that fail for the SPM model, along with the error messages:

In [7]:
import traceback

model = pybamm.lithium_ion.SPM()
model_params = model.get_parameter_info()
for param in model_params:
    if model_params[param][1] == "Parameter":
        params = model.default_parameter_values
        original_param = params[param]
        params[param] = "[input]"
        sim = pybamm.Simulation(model, parameter_values=params)
        try:
            sim.solve([0, 3600], inputs={param: original_param})
        except Exception as e:
            print(f"Failed for parameter {param}. Error was {e}")
            tb = traceback.format_exc()
            print(tb)

Failed for parameter Positive electrode thickness [m]. Error was Cannot interpret 'Addition(-0x41f975ed44957ff1, +, children=['0.000125', 'Positive electrode thickness [m]'], domains={})' as a data type
Traceback (most recent call last):
  File "/tmp/ipykernel_1967968/3514874421.py", line 12, in <module>
    sim.solve([0, 3600], inputs={param: original_param})
  File "/home/mrobins/git/PyBaMM/src/pybamm/simulation.py", line 472, in solve
    self.build(initial_soc=initial_soc, inputs=inputs)
  File "/home/mrobins/git/PyBaMM/src/pybamm/simulation.py", line 328, in build
    self._mesh = pybamm.Mesh(self._geometry, self._submesh_types, self._var_pts)
  File "/home/mrobins/git/PyBaMM/src/pybamm/meshes/meshes.py", line 117, in __init__
    self[domain] = submesh_types[domain](geometry[domain], submesh_pts[domain])
  File "/home/mrobins/git/PyBaMM/src/pybamm/meshes/meshes.py", line 301, in __call__
    return self.submesh_type(lims, npts, **self.submesh_params)
  File "/home/mrobins/git/PyB